In [170]:
# Import necessary libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from unidecode import unidecode
import re
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import random

In [171]:
random_wait = random.randint(1, 3)

#Red flags según artículo
red_flags_auto = ["chocado", "deshuase", "roto", "rota", "dañado", "dañada", "malo", "mala", "repuesto", "repuestos", "no transferible"]

red_flags = red_flags_auto

close_locations = ["Valparaiso", "Viña del Mar"]

In [172]:
login = False #True da más detalles de los autos pero puede ser problemático, False da menos detalles pero es más robusto

# Set up search parameters
radius = 250 # TIENE que ser [1, 2, 5, 10, 20, 40, 60, 80, 100, 250, 500]
search_location = "Valparaiso"
default_location = 111068758922766 #Curacaví, esto permite englobar stgo y valpo
min_price = 1000000
max_price = 10000000
days_since_listed = 30
availability = f"in%20stock" 
# availability = f"out%20of%20stock" #para revisar lo que ya se vendió
min_km = 10000
max_km = 300000
min_year = 1990
max_year = 2020
transmission = "manual"
make = "Mazda "
search_model = "2"
exact = "true" #false busca sugerencias, true busca exactamente lo que se pide

In [173]:
# Configurar el servicio de Chrome
service = Service(ChromeDriverManager().install())

# Configurar las opciones de Chrome
options = Options()
# options.add_argument("--headless")  # Para ejecutar Chrome en modo sin cabeza (sin ventana)
options.add_experimental_option("detach", True) # Para que el driver no se cierre al terminar el script
# deshabilitar notificaciones
options.add_argument("--disable-notifications")

# Inicializar el driver de Chrome
driver = webdriver.Chrome(service=service, options=options)
# Set up base url
base_url = "https://www.facebook.com/marketplace/"

#Set up full url
url = f"{base_url}{default_location}/search?availability={availability}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_since_listed}&maxMileage={max_km}&maxYear={max_year}&minMileage={min_km}&minYear={min_year}&transmissionType={transmission}&query={make}{search_model}&exact={exact}"

In [174]:
# Inicio de sesión o no

# Cuenta fake
mail = "roadtonostromo1@gmail.com"
password = "suufreejvrrr"

if login:

    driver.get(base_url)

    mail_input = driver.find_element(By.XPATH ,'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[3]/div/label/div/div/input')

    ActionChains(driver).click(mail_input).perform()

    mail_input.send_keys(mail)

    password_input = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[2]/form/div/div[4]/div/label/div/div/input')

    ActionChains(driver).click(password_input).perform()

    password_input.send_keys(password)

    mail_input.send_keys(Keys.RETURN)

    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

else:
    # Visitar el sitio web
    driver.get(url)

    # Esperar un tiempo breve para que la página se cargue completamente
    time.sleep(random_wait) # 1 segundo

    # Buscar el elemento de la X para ignorar el inicio de sesión
    x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")

    # Hacer clic en la X para ignorar el inicio de sesión
    x_button.click()

Obtener información de la zona de búsqueda

In [175]:
change_location_selector = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]')

# Obtener ubicación actual
current_location = driver.find_element(By.XPATH, '//*[@id="seo_filters"]/div[1]/div[1]/div/span').text
print(f"La búsqueda se está realizando en: {current_location}")

# Hacer clic en el selector de ubicación
# change_location_selector.click()

# Cambiar la ubicación de búsqueda
# location = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[2]/div/div/div/div/div/label/div/div[2]/input')

# ActionChains(driver).click(location).perform()

# esperar 1 segundo
time.sleep(random_wait)

La búsqueda se está realizando en: Curacaví, Region Metropolitana, Chile
  · En un radio de 65 kilómetros


In [176]:
#location.clear()

In [177]:
#location.send_keys(search_location)

#location.send_keys(Keys.RETURN)

# Hay que hacer que seleccione una de las opciones del menu desplegable

Facebook cambia los id de algunos elementos, cambiar el radio de busqueda como ultima alternativa ya que se debe volver a buscar el XPATH cada cierto tiempo

In [178]:
""" # buscar selector de radio
# radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label')
radius_selector = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div')                                               
# Hacer clic en el selector de radio
radius_selector.click()

# Botones de seleccion de radio

elementos = driver.find_elements(By.XPATH, "//div[@class='x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z']")

# Selecciona el radio en el menú

for elemento in elementos:
    numero = re.match(r"\d+", elemento.text).group()
    
    if int(numero) == radius:
        elemento.click()
        break

# Buscar el botón de aplicar
apply_button = driver.find_element(By.XPATH, '/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div')

# Hacer clic en el botón de aplicar
apply_button.click() """

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
C:\Users\PC\AppData\Local\Temp\ipykernel_15148\3550737752.py:1: SyntaxWarning: invalid escape sequence '\d'
  """ # buscar selector de radio


' # buscar selector de radio\n# radius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label\')\nradius_selector = driver.find_element(By.XPATH, \'/html/body/div[1]/div/div[1]/div/div[4]/div/div/div[1]/div/div[2]/div/div/div/div[3]/div/div[1]/div[3]/div/div/label/div/div[1]/div/div\')                                               \n# Hacer clic en el selector de radio\nradius_selector.click()\n\n# Botones de seleccion de radio\n\nelementos = driver.find_elements(By.XPATH, "//div[@class=\'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n xe8uvvx x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x87ps6o x1lku1pv x1a2a7pz x6s0dn4 xjyslct x9f619 x1ypdohk x78zum5 x1q0g3np x2lah0s xnqzcj9 x1gh759c xdj266r xat24cr x1344otq x1de53dj xz9dl7a xsag5q8 x1n2onr6 x16tdsg8 x1ja2u2z\']")\n\n# Selecciona el radio en el menú\n\nfor elemento in elementos:\n    num

In [179]:
# Desplazarse hacia abajo para cargar más resultados
scroll_count = 6  # Número de veces a desplazar la página
scroll_delay = random_wait  # Retraso entre cada desplazamiento en segundos

for _ in range(scroll_count):
    # Ejecutar JavaScript para desplazarse hasta el final de la página
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Esperar un momento para permitir que se cargue el contenido
    time.sleep(scroll_delay)

In [180]:
# Obtener el HTML de la página
html = driver.page_source

# Crear un objeto BeautifulSoup a partir del HTML
market_soup = soup(html, 'html.parser')

In [181]:
# Extract all the necessary info and insert into lists
titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")
titles_list = [title.text.strip() for title in titles_div]
prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x676frb x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [price.text.strip() for price in prices_div]
locations_and_mileages_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
locations_and_mileages_list = [location_and_mileage.text.strip() for location_and_mileage in locations_and_mileages_div]
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
base_url = "https://web.facebook.com"
urls_list = [base_url+url.get('href') for url in urls_div]

In [182]:
locations_and_mileages_list


['Santiago, RM',
 '94\xa0mil km',
 'Calera, VS',
 '125\xa0mil millas',
 'Padre Hurtado, RM',
 '155\xa0mil km',
 'Viña del Mar, VS',
 '62\xa0mil km',
 'Quilpué, VS',
 '195\xa0mil km',
 'Talagante, RM',
 '184\xa0mil km',
 'Maipú, RM',
 '76\xa0mil km',
 'San Bernardo, RM',
 '76\xa0mil km',
 'Talagante, RM',
 '104\xa0mil km',
 'Puente Alto, RM',
 '198\xa0mil km',
 'Santiago, RM',
 '203\xa0mil km',
 'Conchalí, RM',
 '144\xa0mil km',
 'Viña del Mar, VS',
 '112\xa0mil km',
 'Santiago, RM',
 '83\xa0mil km',
 'Concón, VS',
 '98\xa0mil km',
 'Lo Barnechea, RM',
 '110\xa0mil km',
 'La Reina, RM',
 '154\xa0mil km',
 'Buin, RM',
 '46\xa0mil km',
 'Puente Alto, RM',
 '135\xa0mil km',
 'Ñuñoa, RM',
 '109\xa0mil km',
 'Puente Alto, RM',
 '130\xa0mil km',
 'Viña del Mar, VS',
 '67\xa0mil km',
 'Santiago, RM',
 '139\xa0mil km']

In [183]:
# Create a regular expression pattern to match city and state entries like "City, State"
pattern = re.compile(r'(\w+(?:-\w+)?, [A-Z]{2})')

# Define regular expressions to extract numeric mileage values in "K km" and "K miles" format
mileage_pattern_km = r'(\d+) km'
mileage_pattern_miles = r'(\d+) millas'

# Initialize an empty list to store adjusted mileage entries
locations_and_mileages_list2 = []

# Iterate through the original mileage entries
for item in locations_and_mileages_list:
    # Append the current mileage entry to the adjusted list
    locations_and_mileages_list2.append(item)
    
    """ # Check if the current mileage entry matches the pattern and there are at least two entries in the adjusted list
    if pattern.match(item) and len(locations_and_mileages_list2) >= 2 and pattern.match(locations_and_mileages_list2[-2]):
        # If the conditions are met, insert "0K km" in between the two consecutive city and state entries
        locations_and_mileages_list2.insert(-1, '0\xa0mil km') """
    
    if not item:
        #locations_and_mileages_list2.insert(-1, '0\xa0mil km')
        locations_and_mileages_list2[-1] = item.replace('', '0')
        print("Se encontró un espacio en blanco y se rellenó con 0 km")
    
    if "\xa0mil" in item:
        locations_and_mileages_list2[-1] = item.replace("\xa0mil", "")
        item2 = item.replace("\xa0mil", "")
        #PONER EL CONVERSOR DE MILLAS A KM AQUI

        # Try to find a match for the "K km" format
        match_mileage_km = re.search(mileage_pattern_km, item2)
        
        # Try to find a match for the "K miles" format
        match_mileage_miles = re.search(mileage_pattern_miles, item2)
        
        # Check if either of the formats is found
        if match_mileage_km or match_mileage_miles:
            # If "K km" format is found, convert it to meters and append to the cleaned list
            if match_mileage_km:
                locations_and_mileages_list2[-1] = str((int(match_mileage_km.group(1)) * 1000))
            # If "K miles" format is found, convert it to meters and append to the cleaned list
            else:
                locations_and_mileages_list2[-1] = str((int(match_mileage_miles.group(1)) * 1609))
                print(f"Se encontró una distancia en K millas {int(match_mileage_miles.group(1))}, se convirtió a {int(match_mileage_miles.group(1)) * 1609} metros")

Se encontró una distancia en K millas 125, se convirtió a 201125 metros


In [184]:
locations_and_mileages_list2

['Santiago, RM',
 '94000',
 'Calera, VS',
 '201125',
 'Padre Hurtado, RM',
 '155000',
 'Viña del Mar, VS',
 '62000',
 'Quilpué, VS',
 '195000',
 'Talagante, RM',
 '184000',
 'Maipú, RM',
 '76000',
 'San Bernardo, RM',
 '76000',
 'Talagante, RM',
 '104000',
 'Puente Alto, RM',
 '198000',
 'Santiago, RM',
 '203000',
 'Conchalí, RM',
 '144000',
 'Viña del Mar, VS',
 '112000',
 'Santiago, RM',
 '83000',
 'Concón, VS',
 '98000',
 'Lo Barnechea, RM',
 '110000',
 'La Reina, RM',
 '154000',
 'Buin, RM',
 '46000',
 'Puente Alto, RM',
 '135000',
 'Ñuñoa, RM',
 '109000',
 'Puente Alto, RM',
 '130000',
 'Viña del Mar, VS',
 '67000',
 'Santiago, RM',
 '139000']

In [185]:
# Separate locations and mileages lists

# Initialize an empty list to store cleaned values
mileage_clean = []
locations_list = []

# Iterate through the locations_and_mileages_list2 in steps of 2 to separate locations and mileage
for i in range(0, len(locations_and_mileages_list2), 2):
    # Extract location
    location_parts = locations_and_mileages_list2[i].split(',')
    if len(location_parts) > 0:
        # Clean and decode the location to handle UTF-8 characters properly
        location = location_parts[0].strip().encode('utf-8').decode('utf-8')
        locations_list.append(location)   

    # Extract mileage
    mileage_parts = locations_and_mileages_list2[i + 1].split(' ')
    if len(mileage_parts) > 0:
        mileage = mileage_parts[0]
        mileage_clean.append(int(mileage))

print("Locations:", locations_list)
print("Mileage:", mileage_clean)
print(len(locations_list))
print(len(mileage_clean))

Locations: ['Santiago', 'Calera', 'Padre Hurtado', 'Viña del Mar', 'Quilpué', 'Talagante', 'Maipú', 'San Bernardo', 'Talagante', 'Puente Alto', 'Santiago', 'Conchalí', 'Viña del Mar', 'Santiago', 'Concón', 'Lo Barnechea', 'La Reina', 'Buin', 'Puente Alto', 'Ñuñoa', 'Puente Alto', 'Viña del Mar', 'Santiago']
Mileage: [94000, 201125, 155000, 62000, 195000, 184000, 76000, 76000, 104000, 198000, 203000, 144000, 112000, 83000, 98000, 110000, 154000, 46000, 135000, 109000, 130000, 67000, 139000]
23
23


In [186]:
# Ajustar formato según si está en stock o no
if availability == "out%20of%20stock":
    prices_list =  [elemento for elemento in prices_list if elemento != '·']
#prices_list
    
clean_prices_list = []
# Limpiar y convertir los precios
for precio in prices_list:
    # Remover espacios en blanco y símbolo "$", y convertir a entero
    precio_limpio = int(precio.replace('\xa0', '').replace('$', ''))
    # Agregar el precio limpio a la lista
    clean_prices_list.append(precio_limpio)

In [187]:
def get_description(url):
    driver.get(url)
    
    
    try:
        # Buscar el elemento de la X para ignorar el inicio de sesión
        x_button = driver.find_element(By.XPATH, "//div[@aria-label='Cerrar']")
        # Hacer clic en la X para ignorar el inicio de sesión
        x_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        # time.sleep(random_wait)
    except:
        pass

    try:

        ver_mas_button = driver.find_element(By.XPATH, "//div[contains(@class, 'x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x1n2onr6 x87ps6o x1lku1pv x1a2a7pz')]//*[contains(text(), 'Ver más')]")
        # Hacer clic en el botón "Ver más"
        ver_mas_button.click()
        # Esperar un tiempo breve para que la página se cargue completamente
        

    except:
        pass
    
    html = driver.page_source
    item_soup = soup(html, 'html.parser')

    #description_spans = item_soup.find('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs x1xmvt09 x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x xudqn12 x3x7a5m x6prxxf xvq8zen xo1l8bm xzsf02u")
    description_spans = item_soup.find('div', class_="xz9dl7a x4uap5 xsag5q8 xkhd6sd x126k92a")

    #Si no se encuentra la descripción, se le asigna "No description"
    if description_spans == None:
        description = "No description"
    else:
        description = description_spans.text.strip()

    words = description.split()  # Dividir la descripción en palabras
    if len(words) >= 2 and words[-2:] == ["Ver", "menos"]:
        # Si las últimas dos palabras son "Ver menos", eliminarlas
        description = " ".join(words[:-2]).strip()

    #print(description)
    time.sleep(random_wait)
    return description


La parte de filtrado de resultados obtenidos es la más demorada de todas

In [188]:
# Filter and add all values to a list of dictionaries
items_list = []

patron = re.compile(r'\b{}\b'.format(re.escape(make)), re.IGNORECASE)

for i, item in enumerate(titles_list):
    items_dict = {}
    
    title_split = titles_list[i].split()

    # Check if the title contains the search make
    if re.search(patron, item):

        for flag in red_flags:
            if re.search(r'\b{}\b'.format(re.escape(flag)), item.lower()):
                #if flag == 'detalles' and 'sin detalles' in item.lower():
                if flag == 'detalles' and ('sin detalles' in item.lower() or 'no tiene detalles' in item.lower()):

                    # Si encuentra "detalles" y "sin detalles" está presente en el titulo del anuncio, entonces no es una bandera roja
                    continue
                else:
                    print(f"Se encontró una red flag en el siguiente título: {item}")
                    break
        else:
            items_dict["Titel"] = item
            items_dict["Year"] = int(title_split[0])
            items_dict["Make"] = title_split[1]
            items_dict["Model"] = title_split[2]
            items_dict["Price"] = clean_prices_list[i]
            items_dict["Search Model"] = search_model
            items_dict["Transmission"] = transmission
            items_dict["Mileage"] = mileage_clean[i]
            items_dict["Location"] = locations_list[i]
            items_dict["Description"] = get_description(urls_list[i]) #La parte más lenta
            items_dict["URL"] = urls_list[i]
            

            # Check for red flags in the description
            description = unidecode(items_dict["Description"].lower())  # Convertir la descripción a minúsculas para hacer la búsqueda sin distinción de mayúsculas y minúsculas
            for flag in red_flags:
                if re.search(r'\b{}\b'.format(re.escape(flag)), description):
                    #if flag == 'detalles' and 'sin detalles' in description:
                    if flag == 'detalles' and ('sin detalles' in description or 'no tiene detalles' in description):
                        # Si encuentra "detalles" y "sin detalles" está presente en la descripción, entonces no es una bandera roja
                        continue
                    elif flag == 'detalle' and ('sin detalle' in description or 'ningun detalle' in description):
                        continue
                    else:
                        print(f"Se encontró un red flag en la siguiente descripción: {items_dict['Description']}")
                        break
                    
            else:  
                items_list.append(items_dict)
    
#print(items_list)
print("\nSe obtuvieron "+ str(len(items_list)) + " resultados utilizables")
print("De una búsqueda de "+ str(len(titles_list)) +" resultados")


Se encontró un red flag en la siguiente descripción: Vendo mazda 2 sport año 2011 Radio pantalla Aire acondicionado Dos copias de llaves Papeles al día Tapiz impecable estado Rueda repuesto Auto apañador y económico Solo detalle estético en puerta trasera copiloto que no afecta al cierre de esa puerta

Se obtuvieron 19 resultados utilizables
De una búsqueda de 23 resultados


In [189]:
# Cerrar la sesión del navegador
#driver.quit()

In [190]:
items_df = pd.DataFrame(items_list)

# Set the display option to ensure that all characters in a column are shown
pd.set_option('display.max_colwidth', None)

items_df.tail()

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,URL
14,2012 Mazda 2,2012,Mazda,2,6000000,2,manual,46000,Buin,"Vehículo en excelente estado, poco kilometraje.\nSe vende por viaje. \nEstaré atenta a consultas",https://web.facebook.com/marketplace/item/1778879379190921/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
15,2010 Mazda 2,2010,Mazda,2,4500000,2,manual,135000,Puente Alto,"Mazda 2 sport 1.5 año 2010 color grafito. Vehículo en buenas condiciones, sólo con detalles estéticos. Todas las mantenciones al día, recientemente cambio de embrague, aceite caja de cambios, bieletas y batería. 135.000 kilómetros Doble airbag Cierre centralizado Frenos ABS Aire acondicionado Alza vidrios eléctricos en las 4 puertas 2 llaves, alarma Documentación al día, transferible Precio conversable.",https://web.facebook.com/marketplace/item/373286375627741/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
16,2014 Mazda 2,2014,Mazda,2,5000000,2,manual,130000,Puente Alto,"Se vende en impecable estado mazda 2 con 130.000 km 2 llaves único dueño muy económico, solo por renovación, si no contesto llamadas por favor WhatsApp, se contestan a la brevedad",https://web.facebook.com/marketplace/item/1455246425414616/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
17,2014 Mazda 2,2014,Mazda,2,6900000,2,manual,67000,Viña del Mar,Mazda 2 sport / 1.5 lts / año 2014 / en buen estado y solo 67000 km.,https://web.facebook.com/marketplace/item/1105757940665499/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
18,2013 Mazda 2,2013,Mazda,2,5300000,2,manual,139000,Santiago,"Vendo mazda 2 sport año 2013 1.5 manual en perfecto estado mecánico con 139 mil kilometros por renovación, con sus respectivos detalles estéticos por el año soy el tercer dueño, se puede ver en la comuna de maipu.",https://web.facebook.com/marketplace/item/791452259503637/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD


In [191]:
items_df

,Titel,Year,Make,Model,Price,Search Model,Transmission,Mileage,Location,Description,URL
0,2014 Mazda 2,2014,Mazda,2,5390000,2,manual,94000,Santiago,"AUTO MUY BIEN CUIDADO. MUY ECONÓMICO SIN MULTAS SIN PARTES 100 % TRA SFERIBLE AIRE ACONDICIONADO LLANTAS,CONTROLES VOLANTE. REALMENTE IMPECABLE PERMISO PAGADO 2 COPIAS DE LLAVES",https://web.facebook.com/marketplace/item/2598560920311112/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
1,2012 Mazda mazda2,2012,Mazda,mazda2,3500000,2,manual,201125,Calera,Se vende auto al día sin partes ni multas transferible o se permuta x camioneta en buen estado soy el 3 dueño solo detalles estéticos buen auto. Algo conversable,https://web.facebook.com/marketplace/item/424748983445569/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
2,2012 Mazda 2,2012,Mazda,2,4000000,2,manual,155000,Padre Hurtado,Motor 1.5,https://web.facebook.com/marketplace/item/1146525316364961/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
3,2008 Mazda 2,2008,Mazda,2,4000000,2,manual,195000,Quilpué,"Excelente Vehículo, mantenciones al dia, permiso de circulación hasta 2025 y revisión técnica hasta Febrero del 2025. Detalles minimos y detalles de pintura por los años, nada que afecte el funcionamiento. Tiene Aire Acondicionado, frenos ABS, Doble Airbag. Auto muy Económico 13km/lt ciudad y 18 km/lt Carretera (Calculado) Precio conversable, transferencia inmediata. Cualquier cosa consultar al [hidden information]",https://web.facebook.com/marketplace/item/416551544289316/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
4,2010 Mazda 2,2010,Mazda,2,4500000,2,manual,184000,Talagante,"Soy el tercer dueño y particular. Revision tecnica al dia hasta noviembre del 2024, permiso de circulación pagado hasta marzo 2025, Manutención de aire acondicionado, alternador, Cambio de Bujías, Cambio de embriague hace un año aprox, bieletas nuevas, amortiguadores en buen estado, motor funcionando al 100% sin fugas y muy bien cuidado con todas sus mantenciones, algunos detalles de carroceria por el tiempo, ademas tiene el parabrisas trizado ya que el vehículo se ocupa mucho en carretera, el valor en vitepal es de 90.000 pesos, conversable en el valor, por lo demas es bastante comodo y economico. Cualquier duda o consulta me llaman o escriben.",https://web.facebook.com/marketplace/item/1461740334442770/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
5,2015 Mazda 2,2015,Mazda,2,5780000,2,manual,76000,Maipú,Mazda 2 2015 Motor 1.5cc Mecanico Full equipo Airbags frontales Aire acondicionado Cierre centralizado Alzavidrios eléctricos 76mil km $5.780.000,https://web.facebook.com/marketplace/item/1073578880585281/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
6,2016 Mazda 2,2016,Mazda,2,5600000,2,manual,76000,San Bernardo,VENDO NEW MAZDA 2 SEDAN MOTOR 1.5 AÑO 2016 AL DIA TRANSFERIBLE SIN MULTAS NI PRENDA 2 DUEÑOS BOTON START STOP LLEGAR Y ANDAR $5.600.000 SE HACE NEGOCIO CON PLATA EN MANO SOLO REALES INTERESADOS NO JUEGUEN A COMPRAR!,https://web.facebook.com/marketplace/item/451272010555824/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
7,2016 Mazda 2 sport,2016,Mazda,2,7600000,2,manual,104000,Talagante,"Vendo mi Mazda 2 sport motor 1.5 con caja 6ta mecánico, económico, bastante bien cuidado en términos generales muy buena opción de city car.(similar a Swift , Yaris , río 5 , Accent ) Es la versión con neblineros,radio con pantalla, sensores de estacionamiento, mandos al volante, etc. Tengo las 2 llaves originales también y tiene Neumaticos nuevos y camara de retroceso. Cualquier consulta, al WhatsApp 104.000 kilómetros. Valor $7.600.000-. OJO, en autofact aparecen multas antiguas de TAG, pero esta TOTALMENTE AL DIA, toda deuda cancelada con sus respectivos comprobantes, absolutamente todo en regla como debe ser.",https://web.facebook.com/marketplace/item/322278477068025/?ref=search&referral_code=null&referral_s

In [192]:
# Extraer los precios y URLs de la lista de diccionarios
prices = [item["Price"] for item in items_list]
urls = [item["URL"] for item in items_list]

# Calcular el precio promedio
precio_promedio = np.mean(prices)

# Calcular el precio más barato
precio_mas_barato = np.min(prices)
indice_precio_mas_barato = np.argmin(prices)

# Calcular el percentil 10 más bajo
percentil_10_mas_bajo = np.percentile(prices, 10)

# Calcular la mediana de los precios
mediana_precios = np.median(prices)


# Imprimir los resultados
print("Precio promedio:", precio_promedio)
print("Mediana de precios:", mediana_precios)
print("Percentil 10 más bajo:", percentil_10_mas_bajo)
print("Precio más barato:", precio_mas_barato)
print("URL:", urls[indice_precio_mas_barato])

Precio promedio: 5075000.0
Mediana de precios: 5000000.0
Percentil 10 más bajo: 3997000.0
Precio más barato: 3500000
URL: https://web.facebook.com/marketplace/item/424748983445569/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD
